In [1]:
try:
    from elasticsearch import Elasticsearch
    import pandas as pd
    import json 
    from elasticsearch import helpers 
    import warnings
    warnings.filterwarnings("ignore")
    print("Loaded.....")   
except Exception as e:
    
    print("Some modules are missing{}".format(e))

Loaded.....


In [2]:
df=pd.read_csv("movies.csv")
df1=df.to_dict("records")
df2=pd.read_csv("ratings.csv")

In [3]:
ENDPOINT= "http://localhost:9200/"
es=Elasticsearch(timeout=600,hosts=ENDPOINT)

# DATAFRAME MANIPULATION

In [4]:
df4=df2.groupby(['movieId'])['rating'].mean()
df4

movieId
1         3.872470
2         3.401869
3         3.161017
4         2.384615
5         3.267857
            ...   
161944    5.000000
162376    4.500000
162542    5.000000
162672    3.000000
163949    5.000000
Name: rating, Length: 9066, dtype: float64

In [5]:
df5=pd.merge(df,df4,on="movieId",how="outer")
df5

,movieId,title,genres,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.872470
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.401869
2,3,Grumpier Old Men (1995),Comedy|Romance,3.161017
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.384615
4,5,Father of the Bride Part II (1995),Comedy,3.267857
...,...,...,...,...
9120,162672,Mohenjo Daro (2016),Adventure|Drama|Romance,3.000000
9121,163056,Shin Godzilla (2016),Action|Adventure|Fantasy|Sci-Fi,NaN
9122,163949,The Beatles: Eight Days a Week - The Touring Y...,Documentary,5.000000
9123,164977,The Gay Desperado (1936),Comedy,NaN


In [6]:
which_user=int(input("Tell me your user Id"))

Tell me your user Id1


In [32]:
df6=df2[(df2["userId"]==which_user)]
df6.rename(columns={'rating': 'user_rating'}, inplace=True)
df6.drop("timestamp",axis=1,inplace=True)
df6.drop("userId",axis=1,inplace=True)

In [8]:
df7=pd.merge(df5,df6,on="movieId", how="outer")
df7=df7.fillna(0)
df7

,movieId,title,genres,rating,user_rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.872470,0.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.401869,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,3.161017,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.384615,0.0
4,5,Father of the Bride Part II (1995),Comedy,3.267857,0.0
...,...,...,...,...,...
9120,162672,Mohenjo Daro (2016),Adventure|Drama|Romance,3.000000,0.0
9121,163056,Shin Godzilla (2016),Action|Adventure|Fantasy|Sci-Fi,0.000000,0.0
9122,163949,The Beatles: Eight Days a Week - The Touring Y...,Documentary,5.000000,0.0
9123,164977,The Gay Desperado (1936),Comedy,0.000000,0.0


# DATAFRAME TO JSON

In [9]:
df8=df7.to_dict("records")

def generator2(df8):
    for c,line in enumerate(df8):
        yield{
         '_index':'df8',
         '_type' :'_doc',
         '_id':line.get('movieId'),
         '_source':{
             'title':line.get('title',["No Data"]),
             'genres':line.get('genres',["No Data"]),
             'rating':line.get('rating',["No Data"]),
             'user_rating':line.get('user_rating',["No Data"])
                }
             } 
      #  raise StopIteration
    gen2=generator2(df8)

# Mappings and Settings

In [10]:
 settings = {
    'settings': {
            'number_of_shards': 1,
            'number_of_replicas': 1,
            "similarity": {
             "custom_bm25": {
                "type": "BM25",
                "b": 0,
                "k1" : 0.9
                 
             }
          }
                
            }
    ,
    
    'mappings': {
        
            "properties": {
                "title": {
                    "type": "text",
                    "analyzer": "english"
                },
                "genres":{
                    "type": 'text',
                    "analyzer":"english"
                },
            
             "rating":{
                    "type": "short"
                
                    
             },
                          
          
             "user_rating":{
                    "type": "short"
                    
               } 
            
            }
        
        
           }
    
} 

In [11]:
my=es.indices.create(index="df8",ignore=[400,404],body=settings)

In [12]:
my

{'error': {'root_cause': [{'type': 'resource_already_exists_exception',
    'reason': 'index [df8/td1cOtVqSn-_sjxU4ohh9g] already exists',
    'index_uuid': 'td1cOtVqSn-_sjxU4ohh9g',
    'index': 'df8'}],
  'type': 'resource_already_exists_exception',
  'reason': 'index [df8/td1cOtVqSn-_sjxU4ohh9g] already exists',
  'index_uuid': 'td1cOtVqSn-_sjxU4ohh9g',
  'index': 'df8'},
 'status': 400}

# Insert into Elasticsearch

In [33]:
try:
    res=helpers.bulk(es,generator2(df8))
    print("Working")
except Exception as e:
    print("Done")

Working


In [17]:
user_request=input("What are you looking for?")
query_body = {
  "query": {
      "multi_match":{
          "query":user_request,
          "fields":["title" ,"genres"]
      }
  }
}  
# Pass the query dictionary to the 'body' parameter of the
# client's Search() method, and have it return results:
result = es.search(index="df8", body=query_body,size=10000,)

What are you looking for?batman


# NEW METRIC

In [26]:
def metric(BM25,rating,user_rating):
        if (user_rating>0 and rating >0 ) :         
            return 0.33*(BM25+rating+user_rating)
        elif (user_rating==0 or rating ==0 ): 
            return 0.5*(BM25+rating+user_rating)
        else :
            return BM25

In [27]:
 for i in result["hits"]["hits"]:
    
    BM25_score=i["_score"]*0.5
    rating=i["_source"]["rating"]
    user_rating=i["_source"]["user_rating"]
    new_metric=metric(BM25_score,rating,user_rating)
    i['new_score']=new_metric

result["hits"]["hits"]=sorted(result["hits"]["hits"],key=lambda i:i['new_score'],reverse=True )
    
print ("total hits:", len(result["hits"]["hits"]),"\n")
for i in result['hits']['hits']:
    print(i['_source']['title']+" "+"\n"+i['_source']['genres']+"\n")

total hits: 12 

Batman (1966) 
Action|Adventure|Comedy

Batman (1989) 
Action|Crime|Thriller

Batman Begins (2005) 
Action|Crime|IMAX

Batman: Mask of the Phantasm (1993) 
Animation|Children

Batman: The Dark Knight Returns, Part 1 (2012) 
Action|Animation|Sci-Fi

Batman Returns (1992) 
Action|Crime

Batman: The Dark Knight Returns, Part 2 (2013) 
Action|Animation

Batman Forever (1995) 
Action|Adventure|Comedy|Crime

Batman: Under the Red Hood (2010) 
Action|Animation

Batman & Robin (1997) 
Action|Adventure|Fantasy|Thriller

Batman: Year One (2011) 
Action|Animation|Crime

Batman v Superman: Dawn of Justice (2016) 
Action|Adventure|Fantasy|Sci-Fi

